In [29]:
import pubmed.core as C;reload(C)

<module 'pubmed.core' from '/home/bzhang/DS/PubMedReader/src/main/python/pubmed/core.py'>

In [30]:
a=C.pubMedCitation('/data/pdda_raw/17_pubmed/2016baseline/medline16n0215.xml.gz')

In [31]:
a.peek()

PMID:String                 = 7159390_1
Journal_ISSN:String         = Print_0264-6021
Article_Title:String        = Synthesis of retinyl phosphate mannose and dolichyl phosphate mannose from endogenous and exogenous retinyl phosphate and dolichyl phosphate in microsomal fraction. Specific decrease in endogenous retinyl phosphate mannose synthesis in vitamin A deficiency.
Journal_Title:String        = The Biochemical journal
Journal_Publish_Date:String = 1982-10-15
Journal_Country:String      = ENGLAND
Mesh_Headings:String        = D000818|D002413|D002855|D006224|D003902|D004287|D004288|D066298|D007700|D008297|D008647|D008862|D017830|D011092|D011103|D011105|D011106|D051381|D012710|D014801|D014802
Keywords:String             = null
Abstract:String             = Rat liver microsomal fraction synthesized Ret-P-Man (retinyl phosphate mannose) and Dol-P-Man (dolichyl phosphate mannose) from endogenous Ret-P (retinyl phosphate) and Dol-P (dolichyl phosphate). Ret-P-Man synthesis displayed an 

In [4]:
b=a.where(a.Keywords.isNotNull()).cache()

In [5]:
b.peek()

PMID:String                 = 7159765_1
Journal_ISSN:String         = Print_0091-634X
Article_Title:String        = An English psychiatrist looks at dangerousness.
Journal_Title:String        = The Bulletin of the American Academy of Psychiatry and the Law
Journal_Publish_Date:String = 1982-01-01
Journal_Country:String      = UNITED STATES
Mesh_Headings:String        = D003134|D003416|D003617|D004739|D005239|D006801|D035846|D028642|D010819|D011336|D011379|D011570|D012306|D012935|D014754
Keywords:String             = Legal Approach|Mental Health Act 1959 (Great Britain)|Mental Health Therapies
Abstract:String             = null
Affiliation:String          = null
Author_Identifier:String    = null
LastName:String             = Gunn
ForeName:String             = J
Suffix:String               = null
Initials:String             = J


In [24]:
a1=a.withColumn('Abstract', C.toAscii('Abstract')).cache()

In [25]:
a1.peek()

PMID:String                 = 7159390_1
Journal_ISSN:String         = Print_0264-6021
Article_Title:String        = Synthesis of retinyl phosphate mannose and dolichyl phosphate mannose from endogenous and exogenous retinyl phosphate and dolichyl phosphate in microsomal fraction. Specific decrease in endogenous retinyl phosphate mannose synthesis in vitamin A deficiency.
Journal_Title:String        = The Biochemical journal
Journal_Publish_Date:String = 1982-10-15
Journal_Country:String      = ENGLAND
Mesh_Headings:String        = D000818|D002413|D002855|D006224|D003902|D004287|D004288|D066298|D007700|D008297|D008647|D008862|D017830|D011092|D011103|D011105|D011106|D051381|D012710|D014801|D014802
Keywords:String             = null
Abstract:String             = Rat liver microsomal fraction synthesized Ret-P-Man (retinyl phosphate mannose) and Dol-P-Man (dolichyl phosphate mannose) from endogenous Ret-P (retinyl phosphate) and Dol-P (dolichyl phosphate). Ret-P-Man synthesis displayed an 

In [4]:
b=C.readPubMedXml('/data/pdda_raw/17_pubmed/2016baseline/medline16n0215.xml.gz')

In [5]:
b.printSchema()

root
 |-- Article: struct (nullable = true)
 |    |-- Abstract: struct (nullable = true)
 |    |    |-- AbstractText: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- _Label: string (nullable = true)
 |    |    |    |    |-- _NlmCategory: string (nullable = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |-- ArticleTitle: string (nullable = true)
 |    |-- AuthorList: struct (nullable = true)
 |    |    |-- Author: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- CollectiveName: string (nullable = true)
 |    |    |    |    |-- ForeName: string (nullable = true)
 |    |    |    |    |-- Initials: string (nullable = true)
 |    |    |    |    |-- LastName: string (nullable = true)
 |    |    |    |    |-- Suffix: string (nullable = true)
 |    |    |    |    |-- _ValidYN: string (nullable = true)
 |    |    |-- _CompleteYN: string (nullable = true)
 |

In [6]:
c=b.where(F.col('KeywordList.Keyword').isNotNull()).cache()

In [19]:
c.select('KeywordList.Keyword._VALUE').peek()

AnalysisException: u"cannot resolve 'KeywordList.Keyword[_VALUE]' due to data type mismatch: argument 2 requires integral type, however, '_VALUE' is of string type.;"

In [16]:
def toS(aa):
    return "|".join([e['_VALUE'] for s in aa for e in s]) if isinstance(aa, (list, list)) else None

In [17]:
c.select(F.udf(toS)('KeywordList.Keyword').alias('aS')).peek()

aS:String = Legal Approach|Mental Health Act 1959 (Great Britain)|Mental Health Therapies


In [43]:
from pyspark.sql.utils import AnalysisException
def _getArrCat(_df, path):
    """Return the column in `path`:
    - if it is a simple array, return as string with pip separated
    - if it is a array of array, return as string with pip separated flatten array
    - else return itself
    """
    try:
        _df.select(path)
    except AnalysisException:
        pre, base = path.rsplit('.', 1)
        try:
            _df.select(pre)
        except AnalysisException:
            return F.lit(None).cast('string')
        else:
            _udf = lambda c: [e[base] for s in c for e in s] if isinstance(c, (list, list)) else None
            return F.udf(_udf)(_df[pre]).cast('string')
    else:
        _udf = lambda c: "|".join([str(e) for e in c]) if isinstance(c, list) else c
        return F.udf(_udf)(_df[path]).cast('string')

In [38]:
has_column(c, 'KeywordList.Keyword._VALUE')

True

In [44]:
c.select(_getArrCat(c, 'KeywordList.Keyword._VALUE')).peek()

cast(PythonUDF#<lambda>(KeywordList.Keyword AS Keyword#718) as string):String = [Legal Approach, Mental Health Act 1959 (Great Britain), Mental Health Therapies]


In [37]:
def has_column(df, col):
    try:
        df[col]
        return True
    except AnalysisException:
        return False